#### Cross Asset Allocation
10%-vol cross-asset portfolio consists of both traditional and alternative risk premia.

quarterly rebalancing, no leverage constraints, no transaction costs.

In [2]:
%%capture
%pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org cvxportfolio 

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import scipy.cluster.hierarchy as sch
from scipy.optimize import minimize
from scipy.optimize import Bounds
from scipy.optimize import LinearConstraint
from scipy.optimize import NonlinearConstraint
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import LinearOperator
import cvxportfolio as cvx

In [6]:
map_rp = {
    "ARP_EqLoR":"MSQEFLRG Index", 
    "ARP_EqVal":"MSQEFVAG Index", 
    "ARP_EqMom":"MSQEFMOG Index", 
    "ARP_EqQly":"MSQEFQUG Index", 
    "ARP_FXCry":"SGIFXC10 Index", 
    "ARP_FXVal":"SGIFXM10 Index", 
    "ARP_FIMom":"BXIIXTBP Index", 
    "ARP_FIVal":"BXIISAGU Index", 
    "ARP_FICry":"BXIIEPAG Index", 
    "TRP_Cmdty":"BCOM Index", 
    "TRP_EqLrg":"RIY Index", 
    "TRP_EqSml":"RTY Index", 
    "TRP_EqGEM":"MXEF Index", 
    "TRP_BbgAgg":"LBUSTRUU Index", 
    "TRP_USHY":"LF98TRUU Index", 
    "TRP_CSLL":"CSLLLTOT Index", 
    "TRP_Cembi":"JCBDCOMP Index", 
    "TRP_Embi":"JPEIDIVR Index", 
    "TRP_NexGen":"NGEMCOMP Index", 
    "TRP_GbiEm":"JGENVUUG Index", 
}

In [ ]:
"""
https://stackoverflow.com/questions/68831145/fast-way-to-get-rolling-percentile-ranks
"""
def getPercentile(df, lookback=1000):
    data = df.to_numpy()
    sw = np.lib.stride_tricks.sliding_window_view(data, lookback, axis=0).T
    scores_np = (sw <= sw[-1:, ...]).sum(axis=0).T / sw.shape[0]
    nan_nparray = np.empty((lookback-1, df.shape[1]))
    nan_nparray[:] = np.nan
    scores_np_full = np.insert(scores_np, 0, nan_nparray, axis=0)
    scores_np_df = pd.DataFrame(scores_np_full, columns=df.columns, index=df.index)
    return scores_np_df

In [40]:
"""
Hierarchical Clustering Asset Allocation
Adapted from:
    David Bailey & Marcos Lopez de Prado (2013) "An Open-Source Implementation of the Critical-Line Algorithm for Portfolio Optimization"
    Marcos Lopez de Prado (2016) "Building diversified portfolios that outperform out of sample"
    Thomas Raffinot (2017) "Hierarchical Clustering based Asset Allocation"
    Berowne Hlavaty & Robert Smith (2017) "Post-Modern Portfolio Construction: Examining Recent Innovations in Asset Allocation"
"""

def getHRP(cov, corr):
    # Construct a hierarchical portfolio
    corr, cov = pd.DataFrame(corr), pd.DataFrame(cov)
    dist = correlDist(corr)
    link = sch.linkage(dist, 'single')
    sortIx = getQuasiDiag(link)
    sortIx = corr.index[sortIx].tolist() # recover labels
    hrp = getRecBipart(cov, sortIx)
    return hrp.sort_index()


def getIVP(cov, **kargs):
    # Compute the inverse-variance portfolio
    ivp = 1. / np.diag(cov)
    ivp /= ivp.sum()
    return ivp


def getClusterVar(cov, cItems):
    # Compute variance per cluster
    cov_ = cov.loc[cItems,cItems] # matrix slice
    w_ = getIVP(cov_).reshape(-1,1)
    cVar = np.dot(np.dot(w_.T,cov_),w_)[0,0]
    return cVar


def getQuasiDiag(link):
    # Sort clustered items by distance
    link = link.astype(int)
    sortIx = pd.Series([link[-1,0], link[-1,1]])
    numItems = link[-1,3] # number of original items
    while sortIx.max() >= numItems:
        sortIx.index = range(0, sortIx.shape[0]*2, 2) # make space
        df0 = sortIx[sortIx>=numItems] # find clusters
        i = df0.index
        j = df0.values - numItems
        sortIx[i] = link[j,0] # item 1
        df0 = pd.Series(link[j,1], index=i+1)
        sortIx = pd.concat([sortIx, df0]) #sortIx = sortIx.append(df0) # item 2
        sortIx = sortIx.sort_index() # re-sort
        sortIx.index = range(sortIx.shape[0]) # re-index
    return sortIx.tolist()


def getRecBipart(cov, sortIx):
    # Compute HRP alloc
    w = pd.Series(1,index=sortIx)
    cItems = [sortIx] # initialize all items in one cluster
    while len(cItems) > 0:
        cItems = [i[j:k] for i in cItems 
                  for j,k in ((0,len(i)//2), (len(i)//2,len(i))) 
                  if len(i)>1] # bi-section
        # parse in pairs
        for i in range(0, len(cItems), 2):
            cItems0 = cItems[i] # cluster 1
            cItems1 = cItems[i+1] # cluster 2
            cVar0 = getClusterVar(cov, cItems0)
            cVar1 = getClusterVar(cov, cItems1)
            alpha = 1 - cVar0/(cVar0+cVar1)
            w[cItems0] *= alpha # weight 1
            w[cItems1] *= 1-alpha # weight 2
    return w


def getHARP(cov, sortIx, xReturns=None, riskaversion=0.5, minWt = 0.001, maxWt = 1.0):
    ##### JPM Oct 10 2017, Berowne Hlavaty & Robert Smith #####
    # Compute HAP, HRP and HARP asset allocation
    # xReturns Asset Returns must be rescaled from zero to 1.0! getxprtnnZeroOne(expReturns)
    # riskaversion = Lambda relative importance of risk vs expected returns
    # Based on de Prado’s “getRecBipart” function
    w=pd.Series(1.0/sortIx.__len__() ,index=sortIx)
    wts=pd.DataFrame(data=0,index=sortIx, columns=[0])
    cItems=[sortIx] # initialize all items in one cluster
    wti=0
    while len(cItems)>0:
        cItems=[i[j:k] for i in cItems for j,k in ((0,len(i)//2), (len(i)//2,len(i))) if len(i)>1] # bi-section
        for i in range(0,len(cItems),2): # parse in pairs i=0
            cItems0=cItems[i] # cluster 1
            cItems1=cItems[i+1] # cluster 2
            cVar0=getClusterVar(cov,cItems0) # Single variance number for portfolio 0
            cVar1=getClusterVar(cov,cItems1)
            if xReturns is None:
                alpha=1-cVar0/(cVar0+cVar1)
                w[cItems0]*=alpha # weight 1
                w[cItems1]*=1-alpha # weight 2
            else:
                eVar0 = xReturns[cItems0].values.mean() # Expected Returns of group 1
                eVar1 = xReturns[cItems1].values.mean() # Expected Returns of group 2
                alpha = 1. - cVar0 / (cVar0+cVar1) # HRPi Relative Variance - HRP 'alpha'
                xprtn = 0. + eVar0 / (eVar0+eVar1) # HAPi Expected Return Relative
                # High Risk Aversion results in more stock weight from variance vs. returns.
                w[cItems0] *= riskaversion*alpha + (xprtn*(1.-riskaversion))# HARPi Wt1
                w[cItems1] *= riskaversion*(1.-alpha) + (1.-xprtn)*((1.-riskaversion))#Wt2
        wts[wti] = w #store incremental weights for debugging
        wti +=1
        w[w<minWt] = 0.0
        w = w.clip(0.0, maxWt)
        w[w<maxWt] = w[w<maxWt] / sum(w) # fix rounding errors
    return w


def getClusters(corr = pd.DataFrame, method='ward', bPlot=False):
    ##### JPM Oct 10 2017, Berowne Hlavaty & Robert Smith #####
    # HCP
    corr = corr.replace([np.inf, -np.inf], np.nan).fillna(0, inplace=False)
    clusters = []
    w=pd.Series(1.0,index=corr.index)
    Z = sch.linkage(corr, method) # 'ward', 'single', etc...)
    # Combine raw leaf nodes with linkage matrix
    raw = pd.DataFrame(corr.index, index=corr.index, columns=['cItems0'])
    raw['cItems1'] = np.nan
    raw['Dist'] = 0
    raw['Count'] = 1
    link = pd.DataFrame(Z, columns=['cItems0','cItems1','Dist','Count'])
    linked = pd.concat([raw, link],axis=0, ignore_index=True)
    # Initial Weights
    linked['wt']=1.
    # Traverse linkage in reverse order
    w = linked.__len__()-1
    while w>Z.__len__():
        inWt = linked.loc[w,'wt']/2.0 # weight split
        cItems0, cItems1 = linked.iloc[w,0:2].astype(int)
        linked.loc[[cItems0, cItems1],'wt'] *= inWt
        w-=1
    if bPlot: # Do we need a chart
        fig = PlotDendo(Z, labels=linked.loc[0:Z.__len__(),'wt'].values) # corr.index)
    return linked.loc[0:Z.__len__(),'wt']


def PlotDendo(Z, labels):
    ##### JPM Oct 10 2017, Berowne Hlavaty & Robert Smith #####
    plt.figure(figsize=(8, 8))
    plt.title('Hierarchical Clustering Dendrogram on Correlations')
    # plt.xlabel('sample index')
    # plt.ylabel('distance')
    R = sch.dendrogram(
        Z,
        leaf_rotation=90., # rotates the x axis labels
        leaf_font_size=16., # font size for the x axis labels
        labels=[str(word*100.0) + '%' for word in labels], # label for plot
        color_threshold=None # .5
    )
    ax = plt.gca()
    ax.tick_params(axis='y', labelsize=16)
    plt.show()
    return R


def getxprtnnZeroOne(expReturns=pd.Series):
    ##### JPM Oct 10 2017, Berowne Hlavaty & Robert Smith #####
    # Rescale expected returns from 0 to 1
    if expReturns is None:
        pass
    else:
        idx=expReturns.index
        expReturns = zscore(expReturns)
        expReturns = expReturns/max(abs(expReturns))/2.0+0.5
        expReturns = expReturns / max(abs(expReturns))
        # in case when max(abs()) was -ve, then new max will be < 1.0
        expReturns = pd.Series(expReturns, index=idx)
    return expReturns


def zscore(a, axis=0, ddof=0, keepNaN=False):
    """ NAN Stable Z-Scores"""
    ##### JPM Oct 10 2017, Berowne Hlavaty & Robert Smith #####
    try:
        idx = a.index
    except:
        idx = None
    a = np.asanyarray(a)
    mns = np.nanmean(a, axis=axis)
    sstd = np.nanstd(a=a, axis=axis, ddof=ddof)
    if axis and mns.ndim < a.ndim:
        res = ((a - np.expand_dims(mns, axis=axis)) /
               np.expand_dims(sstd, axis=axis))
    else:
        res = (a - mns) / sstd # RESult
    if not keepNaN:
        res = np.nan_to_num(res) # Default set to zero where was NaN
    res = pd.Series(res, index=idx)
    return res


def correlDist(corr):
    # A distance matrix based on correlation, where 0<=d[i,j]<=1
    # This is a proper distance metric
    return ((1-corr) / 2.) ** .5


def plotCorrMatrix(path, corr, labels=None):
    # Heatmap of the correlation matrix
    if labels is None:
        labels = []
    plt.pcolor(corr)
    plt.colorbar()
    plt.yticks(np.arange(.5, corr.shape[0] + .5), labels)
    plt.xticks(np.arange(.5, corr.shape[0] + .5), labels)
    plt.savefig(path)
    plt.clf()
    plt.close() # reset pylab
    return


def generateData(nobs, size0, size1, sigma1):
    # Time series of correlated variables
    # 1) generating some uncorrelated data
    np.random.seed(seed=12345)
    random.seed(12345)
    x = np.random.normal(0, 1, size=(nobs,size0)) # each row is a variable
    # 2) creating correlation between the variables
    cols = [random.randint(0, size0-1) for i in range(size1)]
    y = x[:,cols] + np.random.normal(0, sigma1, size=(nobs,len(cols)))
    x = np.append(x,y,axis=1)
    x = pd.DataFrame(x, columns=range(1, x.shape[1]+1))
    return x, cols


def getAllocWeights(rtns, exp_rtns=None, riskaversion=0.5):
    rtns.fillna(0)
    if exp_rtns is not None:
        exp_rtns.fillna(0)
    else:
        exp_rtns = rtns.mean()
    cov = rtns.cov()
    corr = rtns.corr()
    dist = correlDist(corr)
    link = sch.linkage(dist, 'average') #'single') #https://towardsdatascience.com/introduction-hierarchical-clustering-d3066c6b560e
    sortIx = getQuasiDiag(link)
    sortIx = corr.index[sortIx].tolist() # recover labels
    ivp = getIVP(cov)
    hrp = getRecBipart(cov, sortIx)
    harp = getHARP(cov, sortIx, exp_rtns, riskaversion)
    hcp = getClusters(corr)
    return ivp, hrp, harp, hcp


def calcAllocWgts(rtns, lookback=260, numRun=0, exp_rtns=None, riskaversion=0.5, naIgnore=True):
    if naIgnore:
        X = rtns
        if exp_rtns is not None:
            EX = exp_rtns
    else:
        X = rtns.dropna(axis=1)
        if exp_rtns is not None:
            EX = exp_rtns[X.columns]
            EX.fillna(method="ffill", inplace=True)
            EX.fillna(method="bfill", inplace=True)
            EX.fillna(value=0, inplace=True)
    if numRun <= 0:
        T = len(X.index)
    else:
        T = lookback + numRun + 1
        X = X[-T:]
        if exp_rtns is not None:
            EX = EX[-T:]
    dfIVP = np.zeros((T,len(X.columns)))
    dfHRP = np.zeros((T,len(X.columns)))
    dfHARP = np.zeros((T,len(X.columns)))
    dfHCP = np.zeros((T,len(X.columns)))
    for t in range(T):
        if t > lookback:
            if exp_rtns is not None:
                ivp_wgt, hrp_wgt, harp_wgt, hcp_wgt = getAllocWeights(X[t-lookback+1:t], EX[t-1:t], riskaversion)
            else:
                ivp_wgt, hrp_wgt, harp_wgt, hcp_wgt = getAllocWeights(X[t-lookback+1:t])
            dfIVP[t] = ivp_wgt
            dfHRP[t] = hrp_wgt
            dfHARP[t] = harp_wgt
            dfHCP[t] = hcp_wgt
    dfIVP = pd.DataFrame(dfIVP,index=X.index,columns=X.columns)
    dfHRP = pd.DataFrame(dfHRP,index=X.index,columns=X.columns)
    dfHARP = pd.DataFrame(dfHARP,index=X.index,columns=X.columns)
    dfHCP = pd.DataFrame(dfHCP,index=X.index,columns=X.columns)
    if not naIgnore:
        dfIVP = dfIVP.reindex(columns=rtns.columns)
        dfHRP = dfIVP.reindex(columns=rtns.columns)
        dfHARP = dfIVP.reindex(columns=rtns.columns)
        dfHCP = dfIVP.reindex(columns=rtns.columns)
    return dfIVP, dfHRP, dfHARP, dfHCP


In [25]:
df_rp = pd.read_csv('cross_asset_allocation_plus_traditional.csv', 
                    index_col=['DATE'], parse_dates=['DATE'])
df_ret = df_rp.pct_change(periods=1,fill_method=None)
df_ret.columns = list(map_rp.keys())
df_ret = df_ret[1:]

In [59]:
df_ivp, df_hrp, df_harp, df_hcp = getAllocWeights(df_ret)
print(list(df_ivp))
print(list(df_hcp))
print(list(df_hrp))
print(list(df_harp))


[0.024307662798539797, 0.04331681708075773, 0.016361859887551667, 0.09106708544039407, 0.030971600365334126, 0.10285574699873355, 0.035499096773310736, 0.1163887277361239, 0.3481195323620805, 0.005290719556838739, 0.005837245403893006, 0.003571429316128669, 0.0032759291885260705, 0.09240981154876056, 0.015809372164622845, 0.02991313717082113, 0.0010544205374466151, 0.017041957665176088, 0.0072709698005083145, 0.009636878204452203]
[0.0625, 0.0625, 0.03125, 0.03125, 0.0625, 0.0625, 0.015625, 0.015625, 0.03125, 0.125, 0.03125, 0.03125, 0.015625, 0.0625, 0.0078125, 0.03125, 0.0625, 0.125, 0.0078125, 0.125]
[0.0017968650804614703, 0.0019824793810478748, 0.001214230369820565, 0.0006517310160058874, 0.0019172125103651772, 0.005008011931733589, 0.0021366737455654856, 0.005783412117137195, 0.014153764803272646, 0.011501466402641047, 0.017063848032767233, 0.0949742215995985, 0.0013387747657244362, 0.35355669480416974, 0.036053545282402656, 0.13987379272475842, 0.05205733932917342, 0.03907284710

C:\Users\tqiu\AppData\Local\Temp\ipykernel_32712\1624920376.py:237: ClusterWarning: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  link = sch.linkage(dist, 'average') #'single') #https://towardsdatascience.com/introduction-hierarchical-clustering-d3066c6b560e
C:\Users\tqiu\AppData\Local\Temp\ipykernel_32712\1624920376.py:69: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.04614585 0.04614585 0.04614585 0.04614585 0.04614585 0.04614585
 0.04614585 0.04614585 0.04614585 0.04614585]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  w[cItems0] *= alpha # weight 1


In [63]:
EQ_Idx = [0,1,2,3,10,11,12]
df_ivp_eq, df_hrp_eq, df_harp_eq, df_hcp_eq = getAllocWeights(df_ret.iloc[:,EQ_Idx])
print(list(df_ivp_eq))
print(list(df_hcp_eq))
print(list(df_hrp_eq))
print(list(df_harp_eq))

[0.12947649931675584, 0.2307301151757658, 0.08715261348280258, 0.48507532474534887, 0.031092503907627446, 0.01902347293699308, 0.01744947043470654]
[0.125, 0.25, 0.0625, 0.0625, 0.125, 0.125, 0.25]
[0.013453220809716952, 0.012712542679853835, 0.007777974792542936, 0.2544290632190139, 0.14277539975631026, 0.08663841387807396, 0.4822133848644882]
[0.14346802382184334, 0.14715606847075008, 0.10971749886359215, 0.1014935781091766, 0.16411580437225576, 0.08859812329520936, 0.2454509030671727]


C:\Users\tqiu\AppData\Local\Temp\ipykernel_32712\1624920376.py:237: ClusterWarning: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  link = sch.linkage(dist, 'average') #'single') #https://towardsdatascience.com/introduction-hierarchical-clustering-d3066c6b560e
C:\Users\tqiu\AppData\Local\Temp\ipykernel_32712\1624920376.py:69: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.03394374 0.03394374 0.03394374]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  w[cItems0] *= alpha # weight 1


In [64]:
FICC_Idx = [4,5,6,7,8,9,13,14,15,16,17,18,19]
df_ivp_ficc, df_hrp_ficc, df_harp_ficc, df_hcp_ficc = getAllocWeights(df_ret.iloc[:,FICC_Idx])
print(list(df_ivp_ficc))
print(list(df_hcp_ficc))
print(list(df_hrp_ficc))
print(list(df_harp_ficc))

[0.03813006329918311, 0.12662878564506375, 0.043703999504823864, 0.14328964288384186, 0.4285803654984927, 0.006513563045527524, 0.11376848216612358, 0.019463390791782526, 0.03682695761105052, 0.001298128652137681, 0.020980863657355044, 0.008951508332457456, 0.01186424891216035]
[0.0625, 0.0625, 0.0625, 0.0625, 0.125, 0.125, 0.125, 0.015625, 0.03125, 0.25, 0.03125, 0.015625, 0.03125]
[0.001605724685919088, 0.42405543281346153, 0.043242574596578334, 0.22891154049649884, 0.05221985769257532, 0.008920450318695167, 0.013416956735037778, 0.01846041423185821, 0.005947245270204813, 0.01017639033906827, 0.004341768022622618, 0.09939821840528862, 0.08930342639219144]
[0.1765695223891541, 0.16891741149073014, 0.09285226163819542, 0.13681614642651294, 0.0, 0.0, 0.07192426269432647, 0.04978546830399817, 0.02742510049348915, 0.06161164794099855, 0.04764106599923073, 0.06960396627123815, 0.09685314635212616]


C:\Users\tqiu\AppData\Local\Temp\ipykernel_32712\1624920376.py:237: ClusterWarning: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  link = sch.linkage(dist, 'average') #'single') #https://towardsdatascience.com/introduction-hierarchical-clustering-d3066c6b560e
C:\Users\tqiu\AppData\Local\Temp\ipykernel_32712\1624920376.py:69: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.75895558 0.75895558 0.75895558 0.75895558 0.75895558 0.75895558]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  w[cItems0] *= alpha # weight 1
